# Klasifikace popisků

__Zadání:__ vytvořit model, který bude na základě partikulární popisků predikovat obecnou kategorii popisku.

Pouze pro nepolitiky.

In [1]:
import psycopg2 as db
import pandas as pd
import os

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# FIXME: insert credentials if you want to run it
POSTGRES_CONNECTION = (
    "dbname='' user='' host='' password=''"
)

con = db.connect(POSTGRES_CONNECTION)
cur = con.cursor()

cur.execute("select id, party_name, descriptions from persons_view where party_name in ('BEZPP', 'ZAHR')")
descriptions = cur.fetchall()


In [3]:
general_descriptions = ['advokát', 'akademik', 'aktivista', 'blogger', 'byznys', 
                'celebrita', 'církev', 'europol', 'komentátor', 'kontrolor', 
                'kultura', 'léčitel', 'lobby', 'zdravotník', 'mluvčí', 
                'ngo', 'novinář', 'odborník', 'odbory', 'ostatní', 'ombudsman', 
                'ozbrojenec', 'politik', 'social', 'soudce', 'soudce_us', 
                'spolek', 'sport', 'starosta', 'stát_byznys', 'ttank', 
                'umělec', 'úředník', 'vyslanec', 'zaměstnavatel', 'zástupce', 
                'záchranář', 'média', 'mezinárodní']

desc = [i[2].split(";") for i in descriptions]

In [4]:
# rozděl popisky na obecné a partikulární

desc_pairs = []
for i in desc:
    general_d = []
    particular_d = []
    for one_description in i:
        if one_description.strip() in general_descriptions:
            general_d = general_d + [one_description.strip()]
        else:
            particular_d = particular_d + [one_description.strip()]
    desc_pairs = desc_pairs + [(general_d, particular_d)]

# odstranit páry, kde není obecný popisek
new_desc_pairs = []
for i in desc_pairs:
    if i[0] != []:
        new_desc_pairs += [(i[0], i[1])]
        
print(len(desc_pairs))
print(len(new_desc_pairs))

42143
41393


In [5]:
new_desc_pairs[0:10]

[(['umělec'], ['dirigent']),
 (['novinář'], ['publicista']),
 (['umělec'], ['grafička a výtvarnice', 'mluvčí Rožnov pod Radhoštěm']),
 (['církev', 'komentátor', 'novinář'],
  ['Reflex', 'starokatolický kněz', 'Svobodné forum']),
 (['umělec'], ['herec']),
 (['mluvčí'], ['mluvčí Královéhradeckého kraje']),
 (['lobby', 'spolek'], ['prezident Asociace Pracovních Agentur']),
 (['byznys'], ['ředitel obchodu a marketingu Tatra Trucks']),
 (['advokát'], ['právník']),
 (['akademik'], ['ČZU'])]

In [6]:
desc_pairs[0:10]

[(['umělec'], ['dirigent']),
 (['novinář'], ['publicista']),
 (['umělec'], ['grafička a výtvarnice', 'mluvčí Rožnov pod Radhoštěm']),
 (['církev', 'komentátor', 'novinář'],
  ['Reflex', 'starokatolický kněz', 'Svobodné forum']),
 (['umělec'], ['herec']),
 (['mluvčí'], ['mluvčí Královéhradeckého kraje']),
 (['lobby', 'spolek'], ['prezident Asociace Pracovních Agentur']),
 (['byznys'], ['ředitel obchodu a marketingu Tatra Trucks']),
 (['advokát'], ['právník']),
 (['akademik'], ['ČZU'])]

In [7]:
desc_pairs = new_desc_pairs

### Heuristická pravidla 

~ přiřazování popisků na základě nalezení substringu



In [8]:
import re

def assign_label_by_grep_one(i):
    if bool(re.search("""farář|biskup|katol|ekumen|evangel|kněz|řehol|kardinál|
    páter|opat|kostel|diecéz|kanovník|františkán|dominikán|kaplan|kapucín|primas""", i)): 
        return "církev"
    elif bool(re.search("""čro|cro|rozhlas|czp|redak|novin|zpravodaj|report|moderátor|
    editor|publicist|komentátor|zprávař|glos""", i)):
        return "novinář"
    elif bool(re.search("^(kultur|kastel|knihov|festival|film|divad|muze|kurátor|výstav|galer|dramatur)", i)):
        return "kultura"
    elif bool(re.search("""^(expert|architek|histor|fyzik|analyt|pilot|porad|veterin|
    makléř|překlad|etnog|ekonom|znalec|znalky|výzkum|věděc|vědky|restaurát)|(log|ložka)$""", i)):
        return "odborník"
    elif bool(re.search("^(armád|vojen|velitel|voják|polic|poruč|strážn|kapitán|generál|náčeln|důstoj|mp|štáb|pčr|kriminal|ačr)", i)):
        return "ozbrojenec"
    elif bool(re.search("^(asocia|spolek|spolk|sdruž|unie|svaz|klub|komor)", i)):
        return "spolek"
    elif bool(re.search("^(av |uk |akadem|katedr|fakult|univer|ústav|vysokošk|muni|děkan|rektor|upol|kvestor)", i)):
        return "akademik"
    elif bool(re.search("^(hráč|útoč|brank|obrán|atlet|basket|volejbal|hokej|fotbal|cykli|závod|házen|tenis|trenér|lyž|florba|rozhodčí|biatlon|cyklokr|futsal|běžkař|bruslař|gymnast|snowboard)", i)):
        return "sport"
    elif bool(re.search("^(právn|advok|obháj|notář)", i)):
        return "advokát"
    elif bool(re.search("^(uměl|skladat|režis|spisovat|zpěv|pěv|herec|hereč|malíř|hudeb|muzik|výtv|dirig|sochař|choreo|foto|housl|pianist|virtuo|perform|básn|taneč|tanec|balet|sbormi|scénár|bavič|písnič|scenár|varhaník|violist|cellist|kinematogr|tenor|soprán|sopran|baryton|scénograf)", i)):
        return "umělec"
    elif bool(re.search("^(zdravot|nemoc|lékář|klinik|primář|přednost|terap|sestra|psychiatr|chirur|zubař|stomatolo|pediatr|geronto)", i)):
        return "zdravotník"
    elif bool(re.search("^(hasič|horsk|hzs|zzs)", i)):
        return "záchranář"
    elif bool(re.search("^(vysl|konzul|diplomat|konsul|honor|chargé|affair|attaché)", i)):
        return "vyslanec"
    elif bool(re.search("^(lesy|sždc|dráh|drah|čd|pošta|pošty|řsd|sshr|diamo|čez|povodí|budvar|řlp|čnb)", i)):
        return "stát_byznys"
    elif bool(re.search("^(prokur|zástup|žalob)", i)):
        return "zástupce"
    elif bool(re.search("^(úřad|úřed|infor|odbor|referen|správ)", i)):
        return "úředník"
    elif bool(re.search("^(sociál|charit|asyl|azyl)", i)):
        return "social"
    else:
        return None


def assign_label_by_grep(x):
    out = set([assign_label_by_grep_one(i.lower()) for i in x])
    out = [i for i in out if i is not None]    
    return out

In [9]:
predicted_labels = [assign_label_by_grep(i[1]) for i in desc_pairs]

In [10]:
actual_labels = [i[0] for i in desc_pairs]

predicted_labels[1:10]


[['novinář'], [], ['církev'], ['umělec'], [], [], [], ['advokát'], []]

In [11]:
actual_labels[1:10]

[['novinář'],
 ['umělec'],
 ['církev', 'komentátor', 'novinář'],
 ['umělec'],
 ['mluvčí'],
 ['lobby', 'spolek'],
 ['byznys'],
 ['advokát'],
 ['akademik']]

In [12]:
# zbavit se případů, pro které heuristika nepredikovala žádný label

predicted_labels_predicted = []
actual_labels_predicted = []
for i in range(0, len(predicted_labels)):
    if len(predicted_labels[i]) > 0:
        predicted_labels_predicted = predicted_labels_predicted + [predicted_labels[i]]
        actual_labels_predicted = actual_labels_predicted + [actual_labels[i]]

print(predicted_labels_predicted[1:10])
print(actual_labels_predicted[1:10])
print(len(predicted_labels_predicted))
print(len(actual_labels_predicted))



[['novinář'], ['církev'], ['umělec'], ['advokát'], ['odborník'], ['umělec'], ['kultura'], ['sport'], ['kultura', 'odborník']]
[['novinář'], ['církev', 'komentátor', 'novinář'], ['umělec'], ['advokát'], ['odborník', 'sport'], ['kultura', 'umělec'], ['kultura'], ['sport'], ['advokát', 'byznys', 'kultura', 'odborník', 'úředník']]
18837
18837


In [13]:
# podíl, pro který heuristika nepredikovala žádný label
print((len(actual_labels) - len(actual_labels_predicted)) / len(actual_labels))

0.5449230546227622


In [14]:
out = []
for f,s in zip(predicted_labels_predicted, actual_labels_predicted):
    for pred in f:
        if pred not in s:
            print(pred, s)
        out = out + [pred in s]

stát_byznys ['úředník']
odborník ['zdravotník']
ozbrojenec ['byznys']
novinář ['sport']
úředník ['kultura']
stát_byznys ['byznys']
spolek ['odborník']
zástupce ['byznys']
úředník ['kultura']
kultura ['odborník']
novinář ['kultura']
umělec ['zdravotník']
úředník ['odborník']
zástupce ['úředník']
odborník ['zdravotník']
zástupce ['byznys']
stát_byznys ['byznys']
odborník ['akademik']
odborník ['kultura']
odborník ['akademik']
novinář ['umělec']
úředník ['odborník']
ozbrojenec ['záchranář']
ozbrojenec ['byznys', 'komentátor', 'novinář', 'sport']
úředník ['kontrolor']
advokát ['ngo']
zástupce ['kultura']
úředník ['akademik']
ozbrojenec ['byznys']
umělec ['kultura']
úředník ['odborník']
úředník ['akademik']
akademik ['advokát']
zástupce ['odborník']
akademik ['odborník']
zástupce ['social']
odborník ['byznys']
zástupce ['odborník']
novinář ['akademik', 'kultura', 'umělec']
kultura ['umělec']
zástupce ['odborník']
advokát ['kontrolor', 'novinář', 'odborník']
akademik ['odborník']
zástupce ['

odborník ['akademik']
stát_byznys ['odborník']
ozbrojenec ['záchranář']
úředník ['byznys', 'odborník']
umělec ['akademik']
zástupce ['ostatní']
zástupce ['byznys']
odborník ['akademik']
umělec ['novinář']
odborník ['novinář']
odborník ['akademik']
spolek ['byznys']
zástupce ['kultura']
ozbrojenec ['záchranář']
spolek ['ngo']
spolek ['odborník']
novinář ['umělec']
novinář ['odborník']
advokát ['odborník']
novinář ['kultura']
odborník ['zdravotník']
odborník ['akademik']
stát_byznys ['úředník']
zdravotník ['úředník']
kultura ['byznys', 'umělec']
spolek ['záchranář']
úředník ['odbory']
odborník ['byznys']
úředník ['odborník']
odborník ['politik']
akademik ['kontrolor', 'zdravotník']
spolek ['akademik', 'odborník']
odborník ['novinář']
odborník ['zdravotník']
odborník ['akademik']
odborník ['akademik']
úředník ['odborník']
ozbrojenec ['byznys', 'sport']
ozbrojenec ['záchranář']
kultura ['umělec']
úředník ['kultura']
odborník ['akademik']
spolek ['ngo']
církev ['kultura']
zástupce ['záchran

ozbrojenec ['záchranář']
umělec ['aktivista', 'novinář']
advokát ['aktivista', 'novinář']
ozbrojenec ['stát_byznys']
odborník ['zdravotník']
zástupce ['úředník']
odborník ['akademik', 'spolek', 'sport']
ozbrojenec ['sport']
zástupce ['kultura']
zdravotník ['záchranář']
spolek ['akademik', 'ttank']
novinář ['média']
ozbrojenec ['byznys']
úředník ['odbory']
ozbrojenec ['odborník']
advokát ['mluvčí']
umělec ['akademik', 'kultura', 'novinář']
akademik ['odborník']
ozbrojenec ['záchranář']
ozbrojenec ['vyslanec']
zástupce ['odborník']
odborník ['kultura']
zástupce ['byznys', 'odborník']
odborník ['akademik', 'byznys']
umělec ['kultura', 'odborník']
úředník ['kultura']
odborník ['zdravotník']
církev ['spolek']
zástupce ['kultura']
kultura ['ozbrojenec', 'umělec']
odborník ['blogger', 'byznys', 'komentátor']
úředník ['kultura']
novinář ['kultura']
umělec ['kultura']
akademik ['soudce']
úředník ['kultura']
úředník ['odbory']
sport ['byznys']
úředník ['odborník']
novinář ['zdravotník']
akademik

ozbrojenec ['spolek', 'sport']
umělec ['kultura']
úředník ['odborník']
úředník ['odborník']
novinář ['kultura', 'umělec']
kultura ['umělec']
odborník ['akademik']
odborník ['novinář']
kultura ['novinář']
odborník ['akademik']
umělec ['sport']
akademik ['umělec']
akademik ['blogger', 'social', 'spolek']
úředník ['byznys']
novinář ['byznys', 'odborník']
zástupce ['byznys']
úředník ['odborník']
úředník ['kultura']
novinář ['odborník', 'umělec']
zástupce ['kultura']
ozbrojenec ['odborník', 'úředník']
stát_byznys ['odborník', 'úředník']
zdravotník ['odborník']
úředník ['církev']
ozbrojenec ['záchranář']
akademik ['zdravotník']
kultura ['úředník']
odborník ['byznys']
zástupce ['ozbrojenec']
akademik ['ozbrojenec']
vyslanec ['odborník']
zástupce ['byznys']
odborník ['akademik']
odborník ['zdravotník']
odborník ['zdravotník']
odborník ['akademik']
úředník ['kultura', 'odborník']
odborník ['akademik']
kultura ['novinář']
advokát ['akademik']
odborník ['akademik', 'zdravotník']
odborník ['ttank'

kultura ['umělec']
úředník ['kultura']
zástupce ['odborník']
ozbrojenec ['politik']
novinář ['média']
kultura ['odborník']
novinář ['kultura']
zástupce ['úředník']
zdravotník ['novinář', 'ostatní']
advokát ['církev']
církev ['úředník']
odborník ['kultura']
vyslanec ['odborník']
ozbrojenec ['záchranář']
odborník ['byznys']
zdravotník ['odborník']
stát_byznys ['odborník']
zástupce ['církev', 'social']
ozbrojenec ['kontrolor']
advokát ['odborník']
zástupce ['ozbrojenec']
úředník ['odbory']
ozbrojenec ['sport']
úředník ['odborník']
ozbrojenec ['spolek']
ozbrojenec ['sport']
odborník ['komentátor', 'novinář']
stát_byznys ['úředník']
odborník ['akademik']
odborník ['akademik']
umělec ['kultura']
ozbrojenec ['byznys']
úředník ['odborník']
ozbrojenec ['záchranář']
ozbrojenec ['akademik']
církev ['kultura']
odborník ['akademik']
zástupce ['ozbrojenec']
ozbrojenec ['sport']
ozbrojenec ['sport']
umělec ['kultura']
ozbrojenec ['ngo']
zdravotník ['kontrolor']
ozbrojenec ['sport']
ozbrojenec ['byzny

ozbrojenec ['kultura', 'odborník', 'spolek']
úředník ['social']
stát_byznys ['úředník']
sport ['umělec']
vyslanec ['byznys']
zástupce ['social']
zástupce ['byznys']
kultura ['umělec']
novinář ['umělec']
kultura ['byznys']
ozbrojenec ['spolek']
umělec ['kultura']
spolek ['úředník']
ozbrojenec ['spolek']
odborník ['média', 'novinář']
odborník ['novinář']
zástupce ['ozbrojenec']
odborník ['byznys', 'stát_byznys']
akademik ['zdravotník']
ozbrojenec ['sport']
úředník ['zdravotník']
odborník ['akademik']
spolek ['odborník']
církev ['akademik']
vyslanec ['akademik', 'odborník']
zástupce ['spolek']
spolek ['ttank']
spolek ['ngo', 'politik']
ozbrojenec ['byznys']
úředník ['kultura']


In [15]:
sum(out) / len(predicted_labels_predicted)

0.8997186388490737

In [16]:
actual_target = [] 
target_map = {'advokát': 0, 
              'akademik': 1, 
              'aktivista': 2, 
              'blogger': 3, 
              'byznys': 4,
              'celebrita': 5, 
              'církev': 6, 
              'europol': 7, 
              'komentátor': 8, 
              'kontrolor': 9,
              'kultura': 10,
              'léčitel': 11, 
              'lobby': 12, 
              'zdravotník': 13, 
              'mluvčí': 14,
              'ngo': 15, 
              'novinář': 16, 
              'odborník': 17, 
              'odbory': 18, 
              'ostatní': 19, 
              'ombudsman': 20,
              'ozbrojenec': 21, 
              'politik': 22, 
              'social': 23, 
              'soudce': 24, 
              'soudce_us': 25,
              'spolek': 26, 
              'sport': 27, 
              'starosta': 28, 
              'stát_byznys': 29, 
              'ttank': 30, 
              'umělec': 31, 
              'úředník': 32, 
              'vyslanec': 33, 
              'zaměstnavatel': 34, 
              'zástupce': 35,
              'záchranář': 36, 
              'média': 37, 
              'mezinárodní': 38}

for i in desc_pairs:
    actual_target += [target_map[i[0][0]]]

print(actual_target[0:10])

import numpy as np
actual_target = np.asarray(actual_target)


[31, 16, 31, 6, 31, 14, 12, 4, 0, 1]


# Naïve Bayes model

Postup:  
- vytvořit tf-idf matici
- nafitovat Naïve Bayes klasifikátor

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split  
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

count_vect = CountVectorizer()

particular_desc = [' '.join(i[1]) for i in desc_pairs]

train_counts = count_vect.fit_transform(particular_desc)
print(count_vect.get_feature_names()[200:210])

tfidf_transformer = TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(train_counts)

X_train, X_test, y_train, y_test = train_test_split(train_tfidf, actual_target, test_size=0.2, random_state=0)  

clf = MultinomialNB().fit(X_train, y_train)




['aecg', 'aera', 'aero', 'aerobik', 'aerobiku', 'aerodynamika', 'aerodynamiky', 'aerofilms', 'aeroholding', 'aeroklub']


In [18]:
docs_new = ['kněz chobot', 'fotbalista útočník fk mechov']
new_counts = count_vect.transform(docs_new)
new_tfidf = tfidf_transformer.transform(new_counts)
predicted = clf.predict(new_tfidf)

target_categories = []
for k in target_map.keys():
    target_categories += [k]

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, target_categories[category]))

'kněz chobot' => církev
'fotbalista útočník fk mechov' => sport


In [19]:
# TODO: metriky úspěšnosti klasifikace
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))  

# https://stackoverflow.com/questions/10592605/save-classifier-to-disk-in-scikit-learn



[[ 54   0   0 ...   0   0   0]
 [  0 370   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...  63   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
             precision    recall  f1-score   support

          0       0.98      0.65      0.78        83
          1       0.90      0.80      0.85       465
          2       0.00      0.00      0.00        46
          3       0.00      0.00      0.00         1
          4       0.61      0.72      0.66       734
          5       0.00      0.00      0.00         6
          6       0.99      0.51      0.67       162
          8       0.00      0.00      0.00        18
          9       0.00      0.00      0.00        43
         10       0.66      0.75      0.70       811
         11       0.00      0.00      0.00         3
         12       0.00      0.00      0.00        41
         13       0.85      0.68      0.76       324
         14       0.78      0.15      0.25       172
         15 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
import pickle
f = open('nb_clf_2019-04-02.pickle', 'wb')
pickle.dump(clf, f)
f.close()

g = open('count_vect.pickle', 'wb')
pickle.dump(count_vect, g)
g.close()

h = open('tfidf_transformer.pickle', 'wb')
pickle.dump(tfidf_transformer, h)
h.close()

